# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
# Loading the data and assigning it to a variable name df
df = pd.read_csv('homepage_actions.csv')
# Display the first 5 rows to understand the dataset
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
#  Understand the statistics of the data
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [3]:
# Checking if the data contains missing values
df.isna().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [4]:
# Check the 'group' column to understand the value counts for control and experiment groups
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [5]:
# Getting the number of viewers who viewed and clicked and those who just viewed but did not click
viewers_clicked = df[df['action'] == 'click']['id'].nunique()
viewers_viewed = df[df['action'] == 'view']['id'].nunique()
print(f"Number of viewers who also clicked: {viewers_clicked}")
print(f"Number of viewers who only viewed: {viewers_viewed}")

Number of viewers who also clicked: 1860
Number of viewers who only viewed: 6328


In [6]:
# Number of viewers who clicked and viewed based on the two groups
click_counts = df.groupby(['group', 'action']).size().reset_index(name='count')
print(click_counts)

        group action  count
0     control  click    932
1     control   view   3332
2  experiment  click    928
3  experiment   view   2996


In [7]:
# Group actions by 'id' and count the occurrences of 'click' and 'view'
action_counts = df.groupby(['id', 'action']).size().unstack(fill_value=0)
# Find rows where 'action' is 'click,' but 'view' is not present for the same 'id' to understand anomalies with the data
anomalies = action_counts[(action_counts['click'] > 0) & (action_counts['view'] == 0)]
print(anomalies)

Empty DataFrame
Columns: [click, view]
Index: []


All those clicked viewed

In [8]:
# Check for overlap between control and experiment groups
# Form sets of 'id' values for 'control' and 'experiment' groups
id_control = set(df[df['group'] == 'control']['id'])
id_experiment = set(df[df['group'] == 'experiment']['id'])

# Use intersection of sets to find 'id' values that appear in both 'control' and 'experiment' groups
id_overlap = id_control.intersection(id_experiment)

# Convert the result to a list
id_list_overlap = list(id_overlap)

# Print the 'id' values that overlap
print(id_list_overlap)

[]


The control and experiment groups do not have any overlapping 'id' values, which is a positive sign for the experimental design. This implies that there are no overlaps of data.

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

#### Hypotheses:
H0: The average effectiveness of the experimental homepage is  less or the same as that of the control homepage (H0: μ_e <= μ_c)


Ha: The average effectiveness of the experimental homepage is higher than that of the control homepage (Ha: μ_e > μ_c)

In [9]:
from statsmodels.stats.proportion import proportions_ztest
# Separate the data into control and experiment groups
control_group = df[df['group'] == 'control']
experiment_group = df[df['group'] == 'experiment']

# Calculate the number of clicks and total views for each group
clicks_control = (control_group['action'] == 'click').sum()
views_control = len(control_group)
clicks_experiment = (experiment_group['action'] == 'click').sum()
views_experiment = len(experiment_group)

# Set the significance level (alpha)
alpha = 0.05

# Perform a one-tailed z-test for proportions
z_statistic, p_value = proportions_ztest([clicks_experiment, clicks_control], [views_experiment, views_control], alternative='larger')
z_statistic, p_value

(1.9332699751308076, 0.02660147426144761)

In [10]:
"""
The p-value (0.02660147426144761) is less that the selected significance(alpha) level of 0.05, thus we reject the null 
hypothesis. This implies that the average effectiveness of the experimental homepage is significantly higher than that of
the control homepage.

"""

'\nThe p-value (0.02660147426144761) is less that the selected significance(alpha) level of 0.05, thus we reject the null \nhypothesis. This implies that the average effectiveness of the experimental homepage is significantly higher than that of\nthe control homepage.\n\n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [11]:
#Your code here
p_control = clicks_control / views_control
expected_clicks_experiment = p_control * views_experiment
expected_clicks_experiment

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [12]:
#Your code here
import numpy as np

# Observed number of clicks in the experiment group
observed_clicks_experiment = clicks_experiment

# Calculate expected variance
std_dev = np.sqrt(views_experiment * p_control * (1 - p_control))
std_dev

25.888551746650283

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [13]:
#Your code here
from scipy.stats import norm
z_score = (clicks_experiment - expected_clicks_experiment) / std_dev

# Calculate the p-value
p_value = 1 - norm.cdf(z_score)

p_value

0.0033030672759265522

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes, the p_vaue (0.0033030672759265522) of this test roughly match with the p_value (0.02660147426144761) of the previous statistical test. Both p_value at a significant(alpha) level of 0.05 are statistically significant. Both the initial statistical test and the verification process support the conclusion that the experimental homepage is more effective

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.